In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib
import pickle

# Load the dataset
df = pd.read_csv('malicious_phish.csv')
X = df['url']
y = df['type']

# Preprocess the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = vectorizer.fit_transform(X)

# Save the vectorizer using joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_tfidf, y_one_hot, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# Build the neural network model for multi-class classification
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Softmax for multi-class

# Compile the model with categorical crossentropy as the loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, mode='max')  # Monitor val_accuracy and stop when it stops increasing
history = model.fit(X_train.toarray(), y_train, epochs=100, batch_size=1000, validation_data=(X_val.toarray(), y_val), callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test.toarray(), y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Save the trained model and label encoder using joblib
joblib.dump(label_encoder, 'label_encoder_multiclass.joblib')

# Save the model in .h5 format
model.save('neural_network_model_multiclass.h5')





Epoch 1/100


490/490 [==============================] - 5s 8ms/step - loss: 0.3407 - accuracy: 0.8814 - val_loss: 0.2032 - val_accuracy: 0.9296
Epoch 2/100
490/490 [==============================] - 3s 7ms/step - loss: 0.1962 - accuracy: 0.9321 - val_loss: 0.1824 - val_accuracy: 0.9365
Epoch 3/100
490/490 [==============================] - 4s 7ms/step - loss: 0.1802 - accuracy: 0.9368 - val_loss: 0.1752 - val_accuracy: 0.9390
Epoch 4/100
490/490 [==============================] - 3s 7ms/step - loss: 0.1719 - accuracy: 0.9393 - val_loss: 0.1707 - val_accuracy: 0.9408
Epoch 5/100
490/490 [==============================] - 3s 7ms/step - loss: 0.1660 - accuracy: 0.9409 - val_loss: 0.1681 - val_accuracy: 0.9415
Epoch 6/100
490/490 [==============================] - 3s 7ms/step - loss: 0.1623 - accuracy: 0.9421 - val_loss: 0.1655 - val_accuracy: 0.9420
Epoch 7/100
490/490 [==============================] - 3s 7ms/step - loss: 0.1589 - accuracy: 0.9430 - val_loss: 0.1660 - val_accuracy: 0

C:\Users\FahadKamran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import pandas as pd

# Read the existing CSV file
existing_data = pd.read_csv('malicious_phish.csv')

# Read the new CSV files
benign_df = pd.read_csv('Benign_list_big_final.csv', header=None, names=['url'])
defacement_df = pd.read_csv('DefacementSitesURLFiltered.csv', header=None, names=['url'])
malware_df = pd.read_csv('Malware_dataset.csv', header=None, names=['url'])
phishing_df = pd.read_csv('phishing_dataset.csv', header=None, names=['url'])
spam_df = pd.read_csv('spam_dataset.csv', header=None, names=['url'])

# Add a 'Type' column and assign the corresponding type
benign_df['type'] = 'benign'
defacement_df['type'] = 'defacement'
malware_df['type'] = 'malware'
phishing_df['type'] = 'phishing'
spam_df['type'] = 'spam'

# Concatenate all DataFrames
all_df = pd.concat([existing_data, benign_df, defacement_df, malware_df, phishing_df, spam_df], ignore_index=True)

# Save the merged DataFrame to the CSV file
all_df.to_csv('Combined_dataset.csv', index=False, header=False)


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib
import pickle

csv_files = ['Combined_dataset.csv', 'Webpages_Classification_test_data.csv','Webpages_Classification_train_data.csv']

# Read data from each CSV file and concatenate into a single DataFrame
dfs = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file, header=None, names=['url','type'])
    dfs.append(df)

# Concatenate all DataFrames
df_combined = pd.concat(dfs, ignore_index=True)
#print(df_combined)
# Read the combined DataFrame
X = df_combined['url']
y = df_combined['type']

# Preprocess the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = vectorizer.fit_transform(X)

# Save the vectorizer using joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_tfidf, y_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# Build the neural network model for multi-class classification
model = Sequential()
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Softmax for multi-class

# Compile the model with categorical crossentropy as the loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max')  # Monitor val_accuracy and stop when it stops increasing
history = model.fit(X_train.toarray(), y_train, epochs=100, batch_size=750, validation_data=(X_val.toarray(), y_val), callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test.toarray(), y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Save the trained model and label encoder using joblib
joblib.dump(label_encoder, 'label_encoder_multiclass.joblib')

# Save the model in .h5 format
model.save('neural_network_model_multiclass.h5')





Epoch 1/100


2049/2049 [==============================] - 45s 21ms/step - loss: 0.1604 - accuracy: 0.9497 - val_loss: 0.1117 - val_accuracy: 0.9647
Epoch 2/100
2049/2049 [==============================] - 26s 13ms/step - loss: 0.1069 - accuracy: 0.9656 - val_loss: 0.1034 - val_accuracy: 0.9666
Epoch 3/100
2049/2049 [==============================] - 24s 12ms/step - loss: 0.0989 - accuracy: 0.9681 - val_loss: 0.1012 - val_accuracy: 0.9678
Epoch 4/100
2049/2049 [==============================] - 23s 11ms/step - loss: 0.0944 - accuracy: 0.9693 - val_loss: 0.0990 - val_accuracy: 0.9683
Epoch 5/100
2049/2049 [==============================] - 23s 11ms/step - loss: 0.0912 - accuracy: 0.9702 - val_loss: 0.1004 - val_accuracy: 0.9688
Epoch 6/100
2049/2049 [==============================] - 23s 11ms/step - loss: 0.0891 - accuracy: 0.9709 - val_loss: 0.0985 - val_accuracy: 0.9687
Epoch 7/100
2049/2049 [==============================] - 23s 11ms/step - loss: 0.0875 - accuracy: 0.9714 - val_lo

C:\Users\FahadKamran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
